## IMPORT LIBRARIES

In [1]:
!pip install -U langchain langchain-community langchainhub transformers accelerate huggingface_hub --quiet
!pip install -U sentence-transformers faiss-cpu --quiet
!pip install -U pypdf --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 77.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.

## UPLOAD FILES

In [2]:
from google.colab import files

# Upload menu.pdf, faq.txt, offers.csv
uploaded = files.upload()

Saving faq.txt to faq.txt
Saving menu.pdf to menu.pdf
Saving offers.csv to offers.csv


## LOAD AND COMBINE ALL FILES

In [3]:
from langchain_community.document_loaders import PyPDFLoader, TextLoader, CSVLoader

# Load all three file types
pdf_loader = PyPDFLoader("menu.pdf")
txt_loader = TextLoader("faq.txt")
csv_loader = CSVLoader("offers.csv")

# Load documents
docs = pdf_loader.load() + txt_loader.load() + csv_loader.load()
# print(f"Loaded {len(docs)} documents.")
print(docs)


[Document(metadata={'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'creationdate': '2025-05-14T23:53:41+05:30', 'author': 'ANSH KEDIA', 'moddate': '2025-05-14T23:53:41+05:30', 'source': 'menu.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='Brew Haven Café - Menu \n \n       Hot Beverages \n- Espresso ................... ₹120 \n- Cappuccino ................ ₹150 \n- Café Latte ................ ₹160 \n- Hot Chocolate ............. ₹180 \n- Masala Chai ............... ₹100 \n \n       Cold Beverages \n- Iced Americano ............ ₹130 \n- Cold Coffee ............... ₹150 \n- Iced Lemon Tea ............ ₹120 \n- Oreo Shake ................ ₹200 \n \n     Desserts \n- Chocolate Brownie ......... ₹110 \n- Cheesecake ................ ₹170 \n- Blueberry Muffin .......... ₹90 \n \n        Snacks \n- Veg Sandwich .............. ₹120 \n- Chicken Panini ............ ₹180 \n- Fries ..................... ₹90 \n \n   Vegan Options: Almond Milk, Soy Milk, 

## SPLIT AND EMBED DOCUMENTS

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# Split into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
docs_split = splitter.split_documents(docs)

# Use sentence-transformers to create embeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Store in vector DB
db = FAISS.from_documents(docs_split, embeddings)


<ipython-input-4-ce1ee834d524>:10: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.wa

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## LOAD HUGGING FACE MODEL (FLAN-T5)

In [5]:
from langchain_community.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

# Load FLAN-T5 for Q&A
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_length=256)
llm = HuggingFacePipeline(pipeline=pipe)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cuda:0
<ipython-input-5-6bfbfc04ff54>:10: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


## SETUP RETREIVAL Q-A CHAIN  

In [6]:
from langchain.chains import RetrievalQA

# Create retriever and QA chain
retriever = db.as_retriever(search_kwargs={"k": 3})
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

## ASK QUESTIONS INTERACTIVELY

In [7]:
# Ask questions in a loop
print("✅ Ask your café-related questions below. Type 'exit' to stop.\n")

while True:
    query = input("🔎 Your question: ")
    if query.lower() in ["exit", "quit", "stop"]:
        print("👋 Exiting. Thank you!")
        break
    try:
        response = qa_chain.invoke(query)
        print("💬 Answer:", response["result"], "\n")
    except Exception as e:
        print("❌ Error:", e, "\n")

✅ Ask your café-related questions below. Type 'exit' to stop.

🔎 Your question: What are the offers for today. 
💬 Answer: Weekend Special,Buy 1 Get 1 Free on Cold Coffees (Sat & Sun),2025-07-31 Offer Name,Description,Valid Till: Happy Hours,20% off on all drinks between 4 PM - 6 PM,2025-06-30 Offer Name,Description,Valid Till: Student Discount,10% off with student ID,2025-12-31 

🔎 Your question: What are the vegan options
💬 Answer: Almond Milk, Soy Milk, Vegan Sandwich 

🔎 Your question: What are the drinks available
💬 Answer: Hot Beverages - Espresso ................... 120 - Cappuccino ................ 150 - Café Latte ................ 160 - Hot Chocolate ............. 180 - Masala Chai ............... 100 Cold Beverages - Iced Americano ............ 130 - Cold Coffee ............... 150 - Iced Lemon Tea ............ 120 - Oreo Shake ................ 200 Desserts - Chocolate Brownie ......... 110 - Cheesecake ................ 170 

🔎 Your question: exit
👋 Exiting. Thank you!


In [8]:
import re

def clean_repetition(text):
    # Remove repeated phrases like "Iced Lemon Tea" occurring many times
    return re.sub(r'(\b[\w\s]{1,30}\b)(?:,?\s*\1\b)+', r'\1', text, flags=re.IGNORECASE)

def get_answer_from_qa_chain(question):
    try:
        response = qa_chain.invoke(question)
        clean_response = clean_repetition(response["result"])
        return clean_response
    except Exception as e:
        return f"Error: {e}"


In [9]:
print(get_answer_from_qa_chain("Do you offer vegan options?"))

Yes, we have almond and soy milk. Vegan sandwiches are also available.


In [10]:
db.save_local("faiss_index")